# Project, Parts 2 and 3, Attempt 1
# Rohan Krishnamurthi
# 10/27/2024

In [42]:
#import os
#from os import path
#import sys
#import shutil
#import subprocess

#bin_dir = path.dirname(sys.executable)
#os.environ['PATH'] += os.pathsep + bin_dir
#shutil.which('bedtools')
#subprocess.run(['bedtools', '--help'])


In [1]:
# Load Packages
import pybedtools
import os
#from pybedtools import BedTool
import pandas as pd


In [54]:
pybedtools.set_bedtools_path("/opt/homebrew/bin/bedtools")

In [3]:
os.environ['PATH'] += ':/opt/homebrew/bin/bedtools'

In [4]:
print(os.environ['PATH'])

/opt/homebrew/bin:/opt/homebrew/sbin:/Users/rohankrishnamurthi/anaconda3/bin:/Users/rohankrishnamurthi/anaconda3/condabin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/TeX/texbin:/Applications/quarto/bin:/opt/homebrew/bin/bedtools


In [43]:
import pybedtools
pybedtools.helpers.set_bedtools_path('/opt/homebrew/bin/bedtools')

In [5]:
# Set working directory
os.chdir("/Users/rohankrishnamurthi/Downloads/Maalavika_Pillai_Research/Histone Mark Analysis Project")

In [6]:
# Set list of cell lines
lines = ["A549", "HepG2", "K562", "GM12878", "MCF-7"]

In [7]:
# Define function to get list of files from a folder
def list_files(folder):
    files = []
    for entry in os.listdir(folder):
        if os.path.isfile(os.path.join(folder, entry)):
            files.append(entry)
    return files

In [11]:
# For loop system
for line in lines: #iterate through all cell line folders
    folder = f"tf_ChIP_data/{line}"
    files = list_files(folder) #collect list of bed files within each cell line
    for file in files:
        TF = file[:file.find("_")] #extract the TF string
        csv_path = f"tf_ChIP_data/filtered_files/{line}_{TF}_filtered.csv"
        bed_path_1 = f"tf_ChIP_data/filtered_files/{line}_{TF}_filtered.bed"
        bed_path_2 = f"tf_ChIP_data/processed_ChIP/{line}_{TF}_accessible_processed.bed" #save path
        
        full_file = folder + "/" + file
        df = pd.read_csv(full_file, sep='\t')
        df_filtered = df[df["num"] >= 2] #must contain 2+ files
        df_filtered = df_filtered[df_filtered["../ATAC-seq/consensuspeaks_all_targets_10kb_sorted.bed"] == 1] # must contain last file      
        df_filtered.to_csv(csv_path, index = False)
        
        df_filtered.to_csv(bed_path_1, sep='\t', index=False, header=False) # save as bedfile
        
        bf = pybedtools.BedTool(bed_path_1) # convert filtered file to a usable bedfile
        bf_merged = bf.merge() #MERGE OPERATION
        
        bf_merged.saveas(bed_path_2) #save merged result
        

In [53]:
# Pybedtools stuff


import pybedtools
from pybedtools import bedtool
from pybedtools import settings
from importlib import reload


def _set_bedtools_path(path=""):
    old_path = settings._bedtools_path
    settings._bedtools_path = path
    if old_path != path:
        reload(bedtool)
        reload(pybedtools)
        return True


def _get_bedtools_path():
    return settings._bedtools_path


def _set_R_path(path=""):
    settings._R_path = path
    